<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>

# Deploying a Model for Inference at Production Scale

## 01 - Getting Started

-------

**Table of Contents**

* [Introduction](#introduction)
* [Triton Inference Server](#triton)
* [Setup](#setup)
* [Conclusion](#conclusion)


<a id="introduction"></a>
### Introduction

In this notebook, we will walk through what Triton Inference Server is as well as do some light setup for our lab.

<a id="triton"></a>
### Triton Inference Server

NVIDIA Triton Inference Server simplifies the deployment of AI models at scale in production. Triton is an open-source, inference-serving software that lets teams deploy trained AI models from any framework, from local storage, or from Google Cloud Platform or Azure on any GPU or CPU-based infrastructure, cloud, data center, or edge. One can get started with Triton by pulling the container from the NVIDIA NGC catalog, the hub for GPU-optimized software for deep learning and machine learning that accelerates deployment to development workflows.

The below figure shows the Triton Inference Server high-level architecture. The model repository is a file-system based repository of the models that Triton will make available for inferencing. Inference requests arrive at the server via either HTTP/REST or GRPC or by the C API and are then routed to the appropriate per-model scheduler. Triton implements multiple scheduling and batching algorithms that can be configured on a model-by-model basis. Each model's scheduler optionally performs batching of inference requests and then passes the requests to the backend corresponding to the model type. The backend performs inferencing using the inputs provided in the batched requests to produce the requested outputs. The outputs are then returned.

<img src="./assets/A-schematic-of-Triton-Server-architecture.png" alt="A Schematic of Triton Inference Server" style="width: 600px;"/>

<a id="setup"></a>
### Setup

First, let's check what GPUs we have on our system:

In [1]:
!nvidia-smi

Sun Mar  5 16:20:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   42C    P0    26W /  70W |    174MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

We see our system has 1 GPU, a Tesla T4.

Additionally, let's examine our file system:

In [2]:
!ls -alh

total 184K
drwxrwxrwx 5 root root 4.0K Mar  5 16:21 .
drwxr-xr-x 1 root root 4.0K Jan 10  2022 ..
drwxr-xr-x 2 root root 4.0K Mar  5 16:11 .ipynb_checkpoints
-rw-rw-rw- 1 root root 4.7K Mar  5 16:12 00_jupyterlab.ipynb
-rw-rw-rw- 1 root root 8.2K Mar  5 16:21 01_Getting_Started.ipynb
-rw-rw-rw- 1 root root  28K Jan  8  2022 02_Simple_PyTorch_Model.ipynb
-rw-rw-rw- 1 root root  15K Dec 14  2021 03_HuggingFace_NLP_Model.ipynb
-rw-rw-rw- 1 root root  17K Dec 14  2021 04_Simple_TensorFlow_Model.ipynb
-rw-rw-rw- 1 root root  23K Dec 14  2021 05_Simple_TensorRT_Model.ipynb
-rw-rw-rw- 1 root root  26K Nov 24  2021 06_Advanced_Inference.ipynb
-rw-rw-rw- 1 root root 8.5K Dec 14  2021 07_Metrics.ipynb
drwxrwxrwx 2 root root 4.0K Jan 10  2022 assets
-rw-rw-rw- 1 root root  14K Nov 18  2021 imagenet-simple-labels.json
drwxr-xr-x 2 root root 4.0K Mar  5 16:07 models


We see several folders and Jupyter notebooks. We'll visit each of these notebooks later in the lab. Lastly, let's check which version of CUDA we're working with. We can see from the output below that we're working with CUDA 11.1.

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


**Server**

In this lab, we already have Triton Inference Server instance running. The code to run a Triton Server Instance is shown below. More details can be found in the quickstart and build instructions:

* [Quickstart Documentation](https://github.com/triton-inference-server/server/blob/r20.12/docs/quickstart.md)
* [Build Documentation](https://github.com/triton-inference-server/server/blob/r20.12/docs/build.md)

```
docker run \
  --gpus=1 \
  --ipc=host --rm \
  --shm-size=1g \
  --ulimit memlock=-1 \
  --ulimit stack=67108864 \
  -p 8000:8000 -p 8001:8001 -p 8002:8002 \
  -v /models:/models \
  nvcr.io/nvidia/tritonserver:20.12-py3 \
  tritonserver \
  --model-repository=/models \
  --exit-on-error=false \
  --model-control-mode=poll \
  --repository-poll-secs 30
```

Triton Inference Server container can be found on NGC: https://ngc.nvidia.com/catalog/containers/nvidia:tritonserver

**Client**

We've also installed the Triton Inference Server Client libraries that provide APIs that make it easy to communicate with Triton from your C++ or Python application. Using these libraries you can send either HTTP/REST or GRPC requests to Triton to access all its capabilities: inferencing, status and health, statistics and metrics, model repository management, etc. These libraries also support using system and CUDA shared memory for passing inputs to and receiving outputs from Triton. Examples show the use of both the C++ and Python libraries.

The easiest way to get the Python client library is to use pip to install the `tritonclient` module, as detailed below. For more details on how to download or build the Triton Inference Server Client libraries, you can find the documentation here: https://github.com/triton-inference-server/server/blob/r20.12/docs/client_libraries.md

```
pip install nvidia-pyindex
pip install tritonclient[all]
```


<a id="conclusion"></a>
### Conclusion

In this notebook, we walked through what Triton Inference Server is as well as did some light setup for our lab. Feel free to move onto the next notebook!

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>